In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs
from math import  inf
from docplex.mp.model import Model

In [222]:
#parameters
yields={1:[3,2.5,2,3,2.5,2,3,2.5,2] , 2:[3.6,3,2.4,3.6,3,2.4,3.6,3,2.4] , 3:[24,20,16,24,20,16,24,20,16]}
probability={1:1/3 , 2:1/3 , 3:1/3}
plant=[150,230,260]
buy =[238,210]
sell=[170, 150, 36, 10]

##### گام اول
##### NLDS(1)

In [223]:
from docplex.mp.model import Model
NLDS1=Model(name='NLDS')
x=NLDS1.continuous_var_matrix(range(1,4),range(1,2),name='x',key_format='%s')
teta1=NLDS1.continuous_var(lb=-inf,name='teta1')
NLDS1.add_constraint_(x[1,1]+x[2,1]+x[3,1]<=500)
NLDS1.add_constraint_(teta1==0)
NLDS1.minimize(sum(plant[i-1]*x[i,j]+teta1 for i in range(1,4) for j in range(1,2)))
sol=NLDS1.solve()
NLDS1.print_solution(print_zeros=True)
#sol.get_value_dict(x)


objective: 0.000
  x11=0.000
  x21=0.000
  x31=0.000
  teta1=0.000


#### NLDS(2,k)

In [255]:
pi_NLDS2={}
for s in range(1,4):
    NLDS2=Model(name='NLDS2{}'.format(s))
    #variables
    x=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta2=NLDS2.continuous_var_list(range(1,4),name='teta2',key_format='%s')
    y=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
    w=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS2.add_constraint_(-x[1,2]-x[2,2]-x[3,2]>=-500,'a')
    NLDS2.add_constraint_(y[1,s,2]-w[1,s,2]>=200,'b')
    NLDS2.add_constraint_(y[2,s,2]-w[2,s,2]>=240,'c')
    NLDS2.add_constraint_(-w[3,s,2]-w[4,s,2]>=0,'d')
    NLDS2.add_constraint_(-w[3,s,2]>=-6000,'e')
    NLDS2.add_constraint_(-x[3,2]>=0,'f')
    NLDS2.add_constraint_(teta2[s-1]==0)
    #objective function
    NLDS2.minimize(150*x[1,2]+230*x[2,2]+260*x[3,2]+sum(buy[i-1]*y[i,s,2] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,2] for j in range(1,5))+teta2[s-1])
    sol=NLDS2.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('d'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('e'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('f'))).item())
    pi_NLDS2['NLDS2{}'.format(s)]=simplex
#print simplex multiplires for each problem
pi_NLDS2

solution for: NLDS21
objective: 98000.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: 98000.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: 98000.000
y132 = 200.000
y232 = 240.000


{'NLDS21': [0, 238.0, 210.0, 0, 0, 0],
 'NLDS22': [0, 238.0, 210.0, 0, 0, 0],
 'NLDS23': [0, 238.0, 210.0, 0, 0, 0]}

#### NLDS(3,k)

In [257]:
pi_NLDS3={}
for s in range(1,10):
    NLDS3=Model(name='NLDS3{}'.format(s))
    #variables
    teta3=NLDS3.continuous_var_list(range(1,10),name='teta2',key_format='%s')
    y=NLDS3.continuous_var_cube(range(1,3),range(1,10),range(2,4),name='y',key_format='%s')
    w=NLDS3.continuous_var_cube(range(1,5),range(1,10),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS3.add_constraint_(y[1,s,3]-w[1,s,2]>=200,'a')
    NLDS3.add_constraint_(y[2,s,3]-w[2,s,3]>=240,'b')
    NLDS3.add_constraint_(-w[3,s,3]-w[4,s,3]>=0,'c')
    NLDS3.add_constraint_(-w[3,s,3]>=-6000,'d')
    NLDS3.add_constraint_(teta3[s-1]==0)
    #objective function
    NLDS3.minimize(sum(buy[i-1]*y[i,s,3] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,3] for j in range(1,5))+teta3[s-1])
    sol=NLDS3.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('d'))).item())
    pi_NLDS3['NLDS3{}'.format(s)]=simplex
#print simplex multiplires for each problem
pi_NLDS3

solution for: NLDS31
objective: 98000.000
y113 = 200.000
y213 = 240.000
solution for: NLDS32
objective: 98000.000
y123 = 200.000
y223 = 240.000
solution for: NLDS33
objective: 98000.000
y133 = 200.000
y233 = 240.000
solution for: NLDS34
objective: 98000.000
y143 = 200.000
y243 = 240.000
solution for: NLDS35
objective: 98000.000
y153 = 200.000
y253 = 240.000
solution for: NLDS36
objective: 98000.000
y163 = 200.000
y263 = 240.000
solution for: NLDS37
objective: 98000.000
y173 = 200.000
y273 = 240.000
solution for: NLDS38
objective: 98000.000
y183 = 200.000
y283 = 240.000
solution for: NLDS39
objective: 98000.000
y193 = 200.000
y293 = 240.000


{'NLDS31': [238.0, 210.0, 0, 0],
 'NLDS32': [238.0, 210.0, 0, 0],
 'NLDS33': [238.0, 210.0, 0, 0],
 'NLDS34': [238.0, 210.0, 0, 0],
 'NLDS35': [238.0, 210.0, 0, 0],
 'NLDS36': [238.0, 210.0, 0, 0],
 'NLDS37': [238.0, 210.0, 0, 0],
 'NLDS38': [238.0, 210.0, 0, 0],
 'NLDS39': [238.0, 210.0, 0, 0]}

#### گام دوم dir=back

In [260]:
#t-1=2
T21=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0]])
T22=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0]])
T23=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0]])
pi31=np.array(pi_NLDS3['NLDS31']).reshape(1,4)
pi32=np.array(pi_NLDS3['NLDS32']).reshape(1,4)
pi33=np.array(pi_NLDS3['NLDS33']).reshape(1,4)
E21=(1/3)*np.matmul(pi31,T21)+(1/3)*np.matmul(pi32,T22)+(1/3)*np.matmul(pi33,T23)
print('E21:',E21)
#E22 and E23 will be the same as E21
h31=np.array([200,240,0,-6000]).reshape(4,1)
h32=h31
h33=h31
e21=(1/3)*np.matmul(pi31,h31)+(1/3)*np.matmul(pi32,h32)+(1/3)*np.matmul(pi33,h33)
print('e21:',e21)

E21: [[595. 630.   0.]]
e21: [[98000.]]


#### دور دوم - گام اول dir=Forward

In [174]:
#define cuts
#595x[1,2]+630x[2,2]+teta21>=98000
#remove teta21==0 constraint

#### NLDS(2,k)

In [265]:
pi_NLDS2={}
sigma_NLDS2={}
for s in range(1,4):
    NLDS2=Model(name='NLDS2{}'.format(s))
    #variables
    x=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
    y=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
    w=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS2.add_constraint_(-x[1,2]-x[2,2]-x[3,2]>=-500,'a')
    NLDS2.add_constraint_(y[1,s,2]-w[1,s,2]>=200,'b')
    NLDS2.add_constraint_(y[2,s,2]-w[2,s,2]>=240,'c')
    NLDS2.add_constraint_(-w[3,s,2]-w[4,s,2]>=0,'d')
    NLDS2.add_constraint_(-w[3,s,2]>=-6000,'e')
    NLDS2.add_constraint_(-x[3,2]>=0,'f')
    NLDS2.add_constraint_(595*x[1,2]+630*x[2,2]+teta2[s-1]>=98000,'g')
    #objective function
    NLDS2.minimize(150*x[1,2]+230*x[2,2]+260*x[3,2]+sum(buy[i-1]*y[i,s,2] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,2] for j in range(1,5))+teta2[s-1])
    sol=NLDS2.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('d'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('e'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('f'))).item())
    pi_NLDS2['NLDS2{}'.format(s)]=simplex
    sigma_NLDS2['NLDS2{}'.format(s)]=np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('g'))).item()
#print simplex multiplires for each problem
print(pi_NLDS2)
print(sigma_NLDS2)

solution for: NLDS21
objective: -26500.000
x12 = 500.000
teta21 = -199500.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: -26500.000
x12 = 500.000
teta22 = -199500.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: -26500.000
x12 = 500.000
teta23 = -199500.000
y132 = 200.000
y232 = 240.000
{'NLDS21': [445.0, 238.0, 210.0, 0, 0, 0], 'NLDS22': [445.0, 238.0, 210.0, 0, 0, 0], 'NLDS23': [445.0, 238.0, 210.0, 0, 0, 0]}
{'NLDS21': 1.0, 'NLDS22': 1.0, 'NLDS23': 1.0}


#### دور دوم - گام دوم dir=back

In [266]:
#t-1=1
T11=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0],[0,0,0],[0,0,0]])
T12=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0],[0,0,0],[0,0,0]])
T13=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0],[0,0,0],[0,0,0]])
pi21=np.array(pi_NLDS2['NLDS21']).reshape(1,6)
pi22=np.array(pi_NLDS2['NLDS22']).reshape(1,6)
pi23=np.array(pi_NLDS2['NLDS23']).reshape(1,6)
E11=(1/3)*np.matmul(pi21,T11)+(1/3)*np.matmul(pi22,T12)+(1/3)*np.matmul(pi23,T13)
print('E11:',E11)
#E22 and E23 will be the same as E21
h21=np.array([-500,200,240,0,-6000,0]).reshape(6,1)
h22=h21
h23=h21
e11=(1/3)*np.matmul(pi21,h21)+(1/3)*np.matmul(pi22,h22)+(1/3)*np.matmul(pi23,h23)
print('e11:',e11)

E11: [[1112.5  714.  4200. ]]
e11: [[-124500.]]


In [ ]:
#define cuts
#1112.5x[1,1]+714[2,1]+4200x[3,1]+teta1>=-124500
#remove teta21==0 constraint

#### دور سوم - گام اول dir=Forward

NLDS(1)

In [263]:
NLDS1=Model(name='NLDS')
x=NLDS1.continuous_var_matrix(range(1,4),range(1,2),name='x',key_format='%s')
teta1=NLDS1.continuous_var(lb=-inf,name='teta1')
NLDS1.add_constraint_(-x[1,1]-x[2,1]-x[3,1]>=-500)
NLDS1.add_constraint_(1112.5*x[1,1]+714*x[2,1]+4200*x[3,1]+teta1>=-124500)
NLDS1.minimize(150*x[1,1]+230*x[2,1]+260*x[3,1]+teta1)
sol=NLDS1.solve()
NLDS1.print_solution(print_zeros=True)
#sol.get_value_dict(x)


objective: -2094500.000
  x11=0.000
  x21=0.000
  x31=500.000
  teta1=-2224500.000


In [267]:
pi_NLDS2={}
sigma_NLDS2={}
for s in range(1,4):
    NLDS2=Model(name='NLDS2{}'.format(s))
    #variables
    x=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
    y=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
    w=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS2.add_constraint_(-x[1,2]-x[2,2]-x[3,2]>=-500,'a')
    NLDS2.add_constraint_(y[1,s,2]-w[1,s,2]>=200,'b')
    NLDS2.add_constraint_(y[2,s,2]-w[2,s,2]>=240,'c')
    NLDS2.add_constraint_(yields[3][s-1]*500-w[3,s,2]-w[4,s,2]>=0,'d')
    NLDS2.add_constraint_(-w[3,s,2]>=-6000,'e')
    NLDS2.add_constraint_(-x[3,2]>=0,'f')
    NLDS2.add_constraint_(595*x[1,2]+630*x[2,2]+teta2[s-1]>=98000,'g')
    #objective function
    NLDS2.minimize(150*x[1,2]+230*x[2,2]+260*x[3,2]+sum(buy[i-1]*y[i,s,2] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,2] for j in range(1,5))+teta2[s-1])
    sol=NLDS2.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('d'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('e'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('f'))).item())
    pi_NLDS2['NLDS2{}'.format(s)]=simplex
    sigma_NLDS2['NLDS2{}'.format(s)]=np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('g'))).item()
#print simplex multiplires for each problem
print(pi_NLDS2)
print(sigma_NLDS2)

solution for: NLDS21
objective: -26500.000
x12 = 500.000
teta21 = -199500.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: -26500.000
x12 = 500.000
teta22 = -199500.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: -26500.000
x12 = 500.000
teta23 = -199500.000
y132 = 200.000
y232 = 240.000
{'NLDS21': [445.0, 238.0, 210.0, 0, 0, 0], 'NLDS22': [445.0, 238.0, 210.0, 0, 0, 0], 'NLDS23': [445.0, 238.0, 210.0, 0, 0, 0]}
{'NLDS21': 1.0, 'NLDS22': 1.0, 'NLDS23': 1.0}


In [268]:
pi_NLDS3={}
for s in range(1,10):
    NLDS3=Model(name='NLDS3{}'.format(s))
    #variables
    teta3=NLDS3.continuous_var_list(range(1,10),name='teta2',key_format='%s')
    y=NLDS3.continuous_var_cube(range(1,3),range(1,10),range(2,4),name='y',key_format='%s')
    w=NLDS3.continuous_var_cube(range(1,5),range(1,10),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS3.add_constraint_(500*yields[1][s-1]+y[1,s,3]-w[1,s,2]>=200,'a')
    NLDS3.add_constraint_(y[2,s,3]-w[2,s,3]>=240,'b')
    NLDS3.add_constraint_(-w[3,s,3]-w[4,s,3]>=0,'c')
    NLDS3.add_constraint_(-w[3,s,3]>=-6000,'d')
    NLDS3.add_constraint_(teta3[s-1]==0)
    #objective function
    NLDS3.minimize(sum(buy[i-1]*y[i,s,3] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,3] for j in range(1,5))+teta3[s-1])
    sol=NLDS3.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('d'))).item())
    pi_NLDS3['NLDS3{}'.format(s)]=simplex
#print simplex multiplires for each problem
pi_NLDS3

solution for: NLDS31
objective: 50400.000
y213 = 240.000
solution for: NLDS32
objective: 50400.000
y223 = 240.000
solution for: NLDS33
objective: 50400.000
y233 = 240.000
solution for: NLDS34
objective: 50400.000
y243 = 240.000
solution for: NLDS35
objective: 50400.000
y253 = 240.000
solution for: NLDS36
objective: 50400.000
y263 = 240.000
solution for: NLDS37
objective: 50400.000
y273 = 240.000
solution for: NLDS38
objective: 50400.000
y283 = 240.000
solution for: NLDS39
objective: 50400.000
y293 = 240.000


{'NLDS31': [0, 210.0, 0, 0],
 'NLDS32': [0, 210.0, 0, 0],
 'NLDS33': [0, 210.0, 0, 0],
 'NLDS34': [0, 210.0, 0, 0],
 'NLDS35': [0, 210.0, 0, 0],
 'NLDS36': [0, 210.0, 0, 0],
 'NLDS37': [0, 210.0, 0, 0],
 'NLDS38': [0, 210.0, 0, 0],
 'NLDS39': [0, 210.0, 0, 0]}

#### دور سوم - گام دوم dir=backward

In [269]:
#t-1=2
T21=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0]])
T22=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0]])
T23=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0]])
pi31=np.array(pi_NLDS3['NLDS31']).reshape(1,4)
pi32=np.array(pi_NLDS3['NLDS32']).reshape(1,4)
pi33=np.array(pi_NLDS3['NLDS33']).reshape(1,4)
E21=(1/3)*np.matmul(pi31,T21)+(1/3)*np.matmul(pi32,T22)+(1/3)*np.matmul(pi33,T23)
print('E21:',E21)
#E22 and E23 will be the same as E21
h31=np.array([200,240,0,-6000]).reshape(4,1)
h32=h31
h33=h31
e21=(1/3)*np.matmul(pi31,h31)+(1/3)*np.matmul(pi32,h32)+(1/3)*np.matmul(pi33,h33)
print('e21:',e21)

E21: [[  0. 630.   0.]]
e21: [[50400.]]


In [270]:
#check for optimality
x12=np.array([[500,0,0]]).reshape(3,1)
if e21-np.matmul(E21,x12)>-199500:
    print('optimality condition does not Hold so we continue')

optimality condition does not Hold so we continue


In [271]:
#define cuts
#add constraint below to NLDS(2,k)
#630*x[2,2]+teta2[s-1]>=50400

In [272]:
pi_NLDS2={}
sigma_NLDS2={}
for s in range(1,4):
    NLDS2=Model(name='NLDS2{}'.format(s))
    #variables
    x=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
    y=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
    w=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS2.add_constraint_(-x[1,2]-x[2,2]-x[3,2]>=-500,'a')
    NLDS2.add_constraint_(y[1,s,2]-w[1,s,2]>=200,'b')
    NLDS2.add_constraint_(y[2,s,2]-w[2,s,2]>=240,'c')
    NLDS2.add_constraint_(yields[3][s-1]*500-w[3,s,2]-w[4,s,2]>=0,'d')
    NLDS2.add_constraint_(-w[3,s,2]>=-6000,'e')
    NLDS2.add_constraint_(-x[3,2]+x[1,1]+x[2,1]>=0,'f')
    NLDS2.add_constraint_(595*x[1,2]+630*x[2,2]+teta2[s-1]>=98000,'g')
    NLDS2.add_constraint_(630*x[2,2]+teta2[s-1]>=50400,'h')
    #objective function
    NLDS2.minimize(150*x[1,2]+230*x[2,2]+260*x[3,2]+sum(buy[i-1]*y[i,s,2] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,2] for j in range(1,5))+teta2[s-1])
    sol=NLDS2.solve()
    sol.display()
    simplex=[]
    sigma=[]
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('d'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('e'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('f'))).item())
    sigma.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('g'))).item())
    sigma.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('h'))).item())
    pi_NLDS2['NLDS2{}'.format(s)]=simplex
    sigma_NLDS2['NLDS2{}'.format(s)]=sigma
#print simplex multiplires for each problem
print(pi_NLDS2)
print(sigma_NLDS2)

solution for: NLDS21
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta21 = -214200.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta22 = -214200.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta23 = -214200.000
y132 = 200.000
y232 = 240.000
{'NLDS21': [400.0, 238.0, 210.0, 0, 0, 0], 'NLDS22': [400.0, 238.0, 210.0, 0, 0, 0], 'NLDS23': [400.0, 238.0, 210.0, 0, 0, 0]}
{'NLDS21': [0.9243697478991597, 0.07563025210084029], 'NLDS22': [0.9243697478991597, 0.07563025210084029], 'NLDS23': [0.9243697478991597, 0.07563025210084029]}


In [273]:
#t-1=1
T11=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0],[0,0,0],[0,0,0]])
T12=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0],[0,0,0],[0,0,0]])
T13=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0],[0,0,0],[0,0,0]])
pi21=np.array(pi_NLDS2['NLDS21']).reshape(1,6)
pi22=np.array(pi_NLDS2['NLDS22']).reshape(1,6)
pi23=np.array(pi_NLDS2['NLDS23']).reshape(1,6)

E11=(1/3)*np.matmul(pi21,T11)+(1/3)*np.matmul(pi22,T12)+(1/3)*np.matmul(pi23,T13)
print('E11:',E11)
#E22 and E23 will be the same as E21
h21=np.array([-500,200,240,0,-6000,0]).reshape(6,1)
h22=h21
h23=h21
e11=(1/3)*np.matmul(pi21,h21)+(1/3)*np.matmul(pi22,h22)+(1/3)*np.matmul(pi23,h23)
print('e11:',e11)

E11: [[1000.  714. 4200.]]
e11: [[-102000.]]


In [274]:
#check for optimality
x1=np.array([[0,0,500]]).reshape(3,1)
if e11-np.matmul(E11,x1)>-2224500:
    print('optimality condition does not Hold so we continue')
else:
    print('yep we did it')

optimality condition does not Hold so we continue


In [275]:
#add the cut below and change dir to Forward
#1000x1+714x2+4200x3+teta1>=-102000

In [276]:
NLDS1=Model(name='NLDS')
x=NLDS1.continuous_var_matrix(range(1,4),range(1,2),name='x',key_format='%s')
teta1=NLDS1.continuous_var(lb=-inf,name='teta1')
NLDS1.add_constraint_(-x[1,1]-x[2,1]-x[3,1]>=-500)
NLDS1.add_constraint_(1112.5*x[1,1]+714*x[2,1]+4200*x[3,1]+teta1>=-124500)
NLDS1.add_constraint_(1000*x[1,1]+714*x[2,1]+4200*x[3,1]+teta1>=-102000)
NLDS1.minimize(150*x[1,1]+230*x[2,1]+260*x[3,1]+teta1)
sol=NLDS1.solve()
NLDS1.print_solution(print_zeros=True)
#sol.get_value_dict(x)


objective: -2072000.000
  x11=0.000
  x21=0.000
  x31=500.000
  teta1=-2202000.000


In [278]:
pi_NLDS2={}
sigma_NLDS2={}
for s in range(1,4):
    NLDS2=Model(name='NLDS2{}'.format(s))
    #variables
    x=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
    y=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
    w=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS2.add_constraint_(-x[1,2]-x[2,2]-x[3,2]>=-500,'a')
    NLDS2.add_constraint_(y[1,s,2]-w[1,s,2]>=200,'b')
    NLDS2.add_constraint_(y[2,s,2]-w[2,s,2]>=240,'c')
    NLDS2.add_constraint_(yields[3][s-1]*500-w[3,s,2]-w[4,s,2]>=0,'d')
    NLDS2.add_constraint_(-w[3,s,2]>=-6000,'e')
    NLDS2.add_constraint_(-x[3,2]>=0,'f')
    NLDS2.add_constraint_(595*x[1,2]+630*x[2,2]+teta2[s-1]>=98000,'g')
    NLDS2.add_constraint_(630*x[2,2]+teta2[s-1]>=50400,'h')
    #objective function
    NLDS2.minimize(150*x[1,2]+230*x[2,2]+260*x[3,2]+sum(buy[i-1]*y[i,s,2] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,2] for j in range(1,5))+teta2[s-1])
    sol=NLDS2.solve()
    sol.display()
    simplex=[]
    sigma=[]
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('d'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('e'))).item())
    simplex.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('f'))).item())
    sigma.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('g'))).item())
    sigma.append(np.array(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('h'))).item())
    pi_NLDS2['NLDS2{}'.format(s)]=simplex
    sigma_NLDS2['NLDS2{}'.format(s)]=sigma
#print simplex multiplires for each problem
print(pi_NLDS2)
print(sigma_NLDS2)

solution for: NLDS21
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta21 = -214200.000
y112 = 200.000
y212 = 240.000
solution for: NLDS22
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta22 = -214200.000
y122 = 200.000
y222 = 240.000
solution for: NLDS23
objective: -7600.000
x12 = 80.000
x22 = 420.000
teta23 = -214200.000
y132 = 200.000
y232 = 240.000
{'NLDS21': [400.0, 238.0, 210.0, 0, 0, 0], 'NLDS22': [400.0, 238.0, 210.0, 0, 0, 0], 'NLDS23': [400.0, 238.0, 210.0, 0, 0, 0]}
{'NLDS21': [0.9243697478991597, 0.07563025210084029], 'NLDS22': [0.9243697478991597, 0.07563025210084029], 'NLDS23': [0.9243697478991597, 0.07563025210084029]}


In [280]:
pi_NLDS3={}
for s in range(1,10):
    NLDS3=Model(name='NLDS3{}'.format(s))
    #variables
    teta3=NLDS3.continuous_var_list(range(1,10),name='teta2',key_format='%s')
    y=NLDS3.continuous_var_cube(range(1,3),range(1,10),range(2,4),name='y',key_format='%s')
    w=NLDS3.continuous_var_cube(range(1,5),range(1,10),range(2,4),name='w',key_format='%s')
    #constraints
    NLDS3.add_constraint_(80*yields[1][s-1]+y[1,s,3]-w[1,s,2]>=200,'a')
    NLDS3.add_constraint_(420*yields[2][s-1]+y[2,s,3]-w[2,s,3]>=240,'b')
    NLDS3.add_constraint_(-w[3,s,3]-w[4,s,3]>=0,'c')
    NLDS3.add_constraint_(-w[3,s,3]>=-6000,'d')
    NLDS3.add_constraint_(teta3[s-1]==0)
    #objective function
    NLDS3.minimize(sum(buy[i-1]*y[i,s,3] for i in range(1,3))+
                sum(sell[j-1]*w[j,s,3] for j in range(1,5))+teta3[s-1])
    sol=NLDS3.solve()
    sol.display()
    simplex=[]
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('a'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('b'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('c'))).item())
    simplex.append(np.array(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('d'))).item())
    pi_NLDS3['NLDS3{}'.format(s)]=simplex
#print simplex multiplires for each problem
pi_NLDS3

solution for: NLDS31
objective: 0.000
solution for: NLDS32
objective: 0.000
solution for: NLDS33
objective: 9520.000
y133 = 40.000
solution for: NLDS34
objective: 0.000
solution for: NLDS35
objective: 0.000
solution for: NLDS36
objective: 9520.000
y163 = 40.000
solution for: NLDS37
objective: 0.000
solution for: NLDS38
objective: 0.000
solution for: NLDS39
objective: 9520.000
y193 = 40.000


{'NLDS31': [0, 0, 0, 0],
 'NLDS32': [238.0, 0, 0, 0],
 'NLDS33': [238.0, 0, 0, 0],
 'NLDS34': [0, 0, 0, 0],
 'NLDS35': [238.0, 0, 0, 0],
 'NLDS36': [238.0, 0, 0, 0],
 'NLDS37': [0, 0, 0, 0],
 'NLDS38': [238.0, 0, 0, 0],
 'NLDS39': [238.0, 0, 0, 0]}

In [281]:
#t-1=2
T21=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0]])
T22=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0]])
T23=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0]])
pi31=np.array(pi_NLDS3['NLDS31']).reshape(1,4)
pi32=np.array(pi_NLDS3['NLDS32']).reshape(1,4)
pi33=np.array(pi_NLDS3['NLDS33']).reshape(1,4)
E21=(1/3)*np.matmul(pi31,T21)+(1/3)*np.matmul(pi32,T22)+(1/3)*np.matmul(pi33,T23)
print('E21:',E21)
#E22 and E23 will be the same as E21
h31=np.array([200,240,0,-6000]).reshape(4,1)
h32=h31
h33=h31
e21=(1/3)*np.matmul(pi31,h31)+(1/3)*np.matmul(pi32,h32)+(1/3)*np.matmul(pi33,h33)
print('e21:',e21)

E21: [[357.   0.   0.]]
e21: [[31733.33333333]]
